(데이터 마이닝) 다양한 분석기법 적용해 모델 개발하는 과정... 모델링 // 기법에 관계없이 개발 위해 데이터 미리 준비하면 수월하다

-> 모델링 진행/분석에 앞서 데이터 수집,변형이 필요.. 잘 정리된 데이터 마트 개발하면 보다 효율적이며 신속한 모델링 가능

데이터 마트: 데이터의 한 부분. 특정 사용자가 관심 가지는 데이터들을 담은 비교적 소규모의 데이터 웨어하우스.

데이터 웨어하우스 <-> 데이터 마트 구분 -> 사용자의 기능/제공 범위를 기준으로!

## R reshape 활용한 데이터 마트

데이터 탐색의 용이한 수행을 위해 데이터 셋에 대한 일종의 변형이 필요.. reshape는 데이터 재정렬을 위한 기법 중 하나

 * 밀집화(aggregation)과는 다름.. 밀집화는 복잡한 데이터 축소, 재정렬함(ex)피벗테이블) 데이터 간단해지나 기존 데이터의 정보들이 손실..

-> reshape는 데이터 재정렬을 수행하되 데이터의 모든 정보를 그대로 유지하는 것이 밀집화와의 차이점

install.packages("reshape")

library(reshape)

data(airquality)

head(airquality) # 데이터 뒤에 개수를 지정할 수 있음! ex) head(airquality, 10)

names(airquality) # 데이터의 변수명을 확인할 수 있다.

names(airquality) = tolower(names(airquality))

names(airquality) # 변수명에 대소문자 혼용하는 경우 헷갈릴 수 있기에 소문자로 전부 변환해 변수명 속성 다시 저장하고 출력

#

reshape의 기능인 melt를 이용해 id의 변수를 기준으로 나머지 변수를 variable이라는 데이터로 만든다. 원래 변수값을 이 value에 저장해 변환한다.

결측값(missing data)를 제거하는 옵션인 na.rm을 "TRUE"로 설정...

aqm = melt(airquality, id=c("month","day"), na.rm=TRUE)

#


cast를 이용해 엑셀의 피벗팅처럼 자료를 변환한다. melt된 aqm으로 "~"를 y, x축의 dimension과 measure에 해당되는 변수의 값을 표시한다.

특별히 변수를 선정하지 않았기에 모든 변수에 대해 분리해 ozone, solar.r처럼 표시한다. 

a = cast(aqm, day ~ month ~ variable) # y축은 month, x축은 variable.. 월/일별 variable의 값을 출력

b = cast(aqm, day ~ month, variable) # 위와 달리 ,로 구분.. 이러면 월별 각 변수들의 평균값(mean)을 산출

c = cast(aqm, month ~ . |variable, mean) # y축은 month / 모든 변수 평균 구하고 "|"를 이용해 산출물을 분리해 표시한다. 

d = cast(aqm, month ~ variable, mean, margins=c("grand_row","grand_col") # margin 옵션.. 행/열에 대해 소계를 산출함

e = cast(aqm, day ~ month, mean, subset=variable="ozone") # 모든 데이터 처리X 특정 변수만 처리하기 위해 subset기능으로 ozone에 대한 변수만 처리

f = cast(aqm, month ~ variable, range) # min/max 동시에 표시하는 range는 각 최소/최대값에 "_X1" / "_X2"라는 접미어를 붙인다.

## sqldf를 이용한 분석

install.packages("sqldf")

library(sqldf)

data(iris)

sqldf("select * from iris") # 조회할 내용을 sqldf 함수 "" 안에 sql로 표현

sqldf("select * from iris limit 10") # 특정 행(row)수만 조회하려면 limit 정의하고 행 개수 쓰기. reshape의 head와 같은 기능이다.

 sqldf("select count(*) from iris where Species like 'se%'") # like 사용함. Species에 se가 포함되는 개수 카운트.

## plyr

데이터 분리,처리한 뒤 결합하는 것과 같은 필수적인 데이터 처리기능을 제공함.

apply 함수와 multi-core 함수를 이용하면 for loop를 안 써도 간단하게 처리 가능. plyr는 apply 함수에 기반에 데이터,출력변수 동시에 배열로 치환해 처리함

ply() 함수는 앞에 두 문자를 접두로 가지는데, 첫 문자는 입력하는 데이터 형태이고 나머지는 출력하는 데이터 형태이다.

d = 데이터 프레임(data.frame) // a = 배열(array) // l = 리스트(list)

-> ddply는 데이터 프레임을 입력받아 처리한 후 데이터 프레임으로 출력함. lyply는 리스트를 입력받아서 처리한 후 데이터 프레임으로 출력..

 set.seed(1) # set.seed는 R에서 난수를 생성할 때 값을 일정하게 고정시켜주는 역할을 함. ()안에 숫자 바꾸면 다른 케이스의 결과가 나오지만 난수값은 같다.


   d = data.frame(year=rep(2012:2014, each=6), count=round(runif(9,0,20))) # runif(생성할 난수의 개수, 최소값, 최대값)

print(d)

year라는 변수에 2012~2014까지의 데이터를 6개씩 넣고, count라는 변수에 난수를 생성하는 runif함수와 반올림을 통해 정수화하는 round함수를 이용해 0에서 20사이의 정수 중 9개를 저장한다. 

In [3]:
# 파이썬에서 보기 편하게 끼워맞춤.. 실제로는 R을 통해 print(d)로 깔끔하게 출력됨. 연도별로 6개의 데이터가..
print("   year count\n1  2012     5\n2  2012     7\n3  2012    11\n4  2012    18\n5  2012     4\n6  2012    18")

   year count
1  2012     5
2  2012     7
3  2012    11
4  2012    18
5  2012     4
6  2012    18


ddply(d, "year", function(x) {     # 데이터프레임으로 입출력하는 ddply함수로 sd와 mean의 비율은 cv(변동계수)를 구하는 기능이다.

+mean.count = mean(x$count) # 데이터 프레임 d를 인수로 받고 count의 mean과 sd를 구해 저장한 뒤 cv를 구한다.

+sd.count = sd(x$count) # 결과를 year별로 리턴하면서 데이터 프레임으로 생성한다.

+cv = sd.count/mean.count


+data.frame(cv.count = cv)

+})

In [4]:
# 출력된 형식만 보세요!
print("  year  cv.count\n1 2012 0.5985621\n2 2013 0.4382254\n3 2014 0.3978489")

  year  cv.count
1 2012 0.5985621
2 2013 0.4382254
3 2014 0.3978489


ddply(d, "year", summarise, mean.count = mean(count)) # summarise 옵션 이용시 count 변수에 명령된 평균이나 합을 계산하고 새로 생긴 변수만 보여준다.

In [5]:
# 코드만!
print("  year mean.count\n1 2012   10.50000\n2 2013   11.33333\n3 2014   14.16667\n")

  year mean.count
1 2012   10.50000
2 2013   11.33333
3 2014   14.16667



ddply(d, "year", transform, total.count = sum(count)) # transform 옵션은 summarise 옵션과는 달리 계산에 사용된 변수도 보여준다.

In [6]:
# 말 안해도..아시죠?
print("   year count total.count\n1  2012     5          63\n2  2012     7          63\n3  2012    11          63\n4  2012    18          63\n5  2012     4          63\n6  2012    18          63\n")

   year count total.count
1  2012     5          63
2  2012     7          63
3  2012    11          63
4  2012    18          63
5  2012     4          63
6  2012    18          63



위의 경우 역시 2012~2014까지 각 해마다 6개의 케이스가 있다.

## 데이터 테이블

data.table.. 데이터 프레임과 유사하지만 보다 빠른 그룹화,순서화,짧은 문장 지원에서 데이터 프레임보다 매력적이지만 무조건 빠르지는 않음.

64비트 환경에서 RAM이 충분할 경우 효율적임.

install.packages("data.table")

library(data.table)

DT = data.table(x=c("b","b","b","a","a"),v=rnorm(5)) # rnorm(5)는 정규분포에서 5개의 난수를 생성한다.

DT


In [9]:
# 출력만
print("   x            v\n1: b -1.539950042\n2: b -0.928567035\n3: b -0.294720447\n4: a -0.005767173\n5: a  2.404653389")

   x            v
1: b -1.539950042
2: b -0.928567035
3: b -0.294720447
4: a -0.005767173
5: a  2.404653389


 data(cars)


head(cars)


In [10]:
# 출력만 보세요
print("  speed dist\n1     4    2\n2     4   10\n3     7    4\n4     7   22\n5     8   16\n6     9   10\n")

  speed dist
1     4    2
2     4   10
3     7    4
4     7   22
5     8   16
6     9   10



 CARS <- data.table(cars) # 데이터프레임으로 된 데이터를 데이터테이블 형식으로 불러와 별도의 이름으로 저장한다.

head(CARS)

In [12]:
# 출력
print("   speed dist\n1:     4    2\n2:     4   10\n3:     7    4\n4:     7   22\n5:     8   16\n6:     9   10")

   speed dist
1:     4    2
2:     4   10
3:     7    4
4:     7   22
5:     8   16
6:     9   10


tables() 

In [14]:
# 출력만
print("   NAME NROW NCOL MB       COLS KEY\n1: CARS   50    2  0 speed,dist    \n2:   DT    5    2  0        x,v\nTotal: 0MB")

   NAME NROW NCOL MB       COLS KEY
1: CARS   50    2  0 speed,dist    
2:   DT    5    2  0        x,v
Total: 0MB


sapply(CARS,class)

In [21]:
print("    speed      dist\n  numeric  numeric ")

    speed      dist
  numeric  numeric 


DT

In [22]:
# 출력만
print("   x            v\n1: b -1.539950042\n2: b -0.928567035\n3: b -0.294720447\n4: a -0.005767173\n5: a  2.404653389")

   x            v
1: b -1.539950042
2: b -0.928567035
3: b -0.294720447
4: a -0.005767173
5: a  2.404653389


DT[2,] # 2번째 행 조회

In [23]:
# 출력만
print("   x         v\n1: b -0.928567")

   x         v
1: b -0.928567


DT[DT$x=="b",] # x값이 b인 데이터 조회

In [24]:
# 출력
print("   x          v\n1: b -1.5399500\n2: b -0.9285670\n3: b -0.2947204")

   x          v
1: b -1.5399500
2: b -0.9285670
3: b -0.2947204


setkey(DT,x) # 데이터 테이블에 key를 지정해 해당 값으로 표시하기

DT

In [25]:
# 출력
print("   x            v\n1: a -0.005767173\n2: a  2.404653389\n3: b -1.539950042\n4: b -0.928567035\n5: b -0.294720447")

   x            v
1: a -0.005767173
2: a  2.404653389
3: b -1.539950042
4: b -0.928567035
5: b -0.294720447


tables() # 데이터 프레임에서와 달리 표시 순서가 x에 의해 ordering되었고 tables()에서 key가 x로 설정되었으며 전체 용량이 2MB임을 알 수 있다.

In [26]:
# 출력
print("   NAME NROW NCOL MB       COLS KEY\n1: CARS   50    2  0 speed,dist    \n2:   DT    5    2  0        x,v   x\nTotal: 0MB")

   NAME NROW NCOL MB       COLS KEY
1: CARS   50    2  0 speed,dist    
2:   DT    5    2  0        x,v   x
Total: 0MB


"b"가 들어간 모든 데이터를 표시하고 싶은 경우, 첫 번째 결과, 마지막 결과, 모든 결과를 표시하는 옵션을 보여주려면..

DT["b",] # 검색 조건 뒤의 , 는 선택사항이다. DT["b"]와 같다.

In [27]:
# 출력
print("   x          v\n1: b -1.5399500\n2: b -0.9285670\n3: b -0.2947204")

   x          v
1: b -1.5399500
2: b -0.9285670
3: b -0.2947204


DT["b",mult="first"] # 첫 번째 결과

In [28]:
# 출력
print("   x        v\n1: b -1.53995")

   x        v
1: b -1.53995


DT["b",mult="last"] # 마지막 결과

In [29]:
# 출력
print("   x          v\n1: b -0.2947204")

   x          v
1: b -0.2947204


4개의 칼럼으로 이루어진 1000만건 자료의 데이터 프레임을 만들자!

grpsize <- ceiling(1e7/26^2) # 천만개의 행과 676개 그룹

tt <- system.time(DF <- data.frame(

+x=rep(LETTERS, each=26*grpsize),

+y=rep(letters, each=grpsize),

+v=runif(grpsize*26^2),

+stringAsFactors=FALSE)


+)

tt

In [31]:
#출력
print("사용자  시스템 elapsed\n0.83    0.09    0.95 ")

사용자  시스템 elapsed
0.83    0.09    0.95 


head(DF,3) # 앞 3줄

In [32]:
# 출력
print("  x y         v stringAsFactors\n1 A a 0.7774452           FALSE\n2 A a 0.9347052           FALSE\n3 A a 0.2121425           FALSE")

  x y         v stringAsFactors
1 A a 0.7774452           FALSE
2 A a 0.9347052           FALSE
3 A a 0.2121425           FALSE


 tail(DF,3) # 뒤 3줄

In [33]:
# 출력
print("         x y         v stringAsFactors\n10000066 Z z 0.1753029           FALSE\n10000067 Z z 0.9537745           FALSE\n10000068 Z z 0.6654964           FALSE")

         x y         v stringAsFactors
10000066 Z z 0.1753029           FALSE
10000067 Z z 0.9537745           FALSE
10000068 Z z 0.6654964           FALSE


dim(DF) # 행,열 개수 조회

[1] 10000068        4 # 3,009,349 rows per seconds

10000068/3.323


[1] 3009349


 tt <- system.time(ans1 <-DF[DF$x=="R" & DF$y=="h",])

tt

In [36]:
# 출력
print("사용자  시스템 elapsed\n0.31    0.06    0.38 ")

사용자  시스템 elapsed
0.31    0.06    0.38 


x는 R, y는 h를 가지는 데이터 프레임 DF에서 자료를 찾는 데 0.38초가 걸렸음을 보였다. 1000만건에서 특정 조건을 만족하는 데이터를 메모리에서 검색해 보였지만 하나하나 비교해 찾는 벡터 검색 방식이라 매우 비효율적임.

head(ans1,3)


In [37]:
# 출력
print("        x y         v stringAsFactors\n6642058 R h 0.5829049           FALSE\n6642059 R h 0.2442074           FALSE\n6642060 R h 0.6491902           FALSE")

        x y         v stringAsFactors
6642058 R h 0.5829049           FALSE
6642059 R h 0.2442074           FALSE
6642060 R h 0.6491902           FALSE


dim(ans1)

[1] 14793     4


이를 데이터 테이블로 변환해 같은 그룹을 추출해보자.

 DT <- data.table(DF)

setkey(DT,x,y)


ss <- system.time(ans2<-DT[J("R","h")]) # binary search

 head(ans2,3)

In [38]:
# 출력
print("   x y         v stringAsFactors\n1: R h 0.5829049           FALSE\n2: R h 0.2442074           FALSE\n3: R h 0.6491902           FALSE")

   x y         v stringAsFactors
1: R h 0.5829049           FALSE
2: R h 0.2442074           FALSE
3: R h 0.6491902           FALSE


dim(ans2)

[1] 14793     4

identical(ans1$ v, ans2$v)


[1] TRUE


ss

In [40]:
# 출력
print(" 사용자  시스템 elapsed\n    0       0       0 ")

 사용자  시스템 elapsed
    0       0       0 


책에서의 예시로는 0.004초가 걸리는 것으로 나온다... 앞선 방식보다는 훨씬 빠를 것임. 하나하나 비교하는 벡터 검색 방식이 아닌 인덱스를 이용하는 바이너리 검색이기 때문! 그렇지만 데이터 테이블을 데이터 프레임처럼 사용하면 성능은 비슷해진다.. 테이블이라고 무조건 빠른것만은 아니다.

### 데이터 테이블을 잘못 사용하는 경우

system.time(ans2 <-DF[DF$x=="R" & DF$y=="h",])

In [42]:
# 출력
print("사용자  시스템 elapsed\n  0.25   0.03   0.33 ")

사용자  시스템 elapsed
  0.25   0.03   0.33 


mapply(identical, ans1, ans2)

In [43]:
# 출력
print("              x               y               v stringAsFactors\n            TRUE            TRUE            TRUE            TRUE ")

              x               y               v stringAsFactors
            TRUE            TRUE            TRUE            TRUE 


지금까지는 DT[...]안의 첫 인수인 i에 대해 공부했다. 이제부터 2/3번째 인수인 j와 by도 알아보자.

### 데이터 테이블에서 summary를 하는 경우

보통 데이터 테이블에서 summary를 하면..

DT[,sum(v)]


[1] 4999770


이번에는 x를 기준으로 summary해보자.

DT[,sum(v), by=x]


In [44]:
# 와 길다 길어
print("    x       V1\n 1: A 192270.8\n 2: B 192261.3\n 3: C 192292.7\n 4: D 191923.5\n 5: E 192457.8\n 6: F 192240.5\n 7: G 192407.9\n 8: H 191992.1\n 9: I 192348.0\n10: J 192436.4\n11: K 192248.5\n12: L 192270.7\n13: M 192280.0\n14: N 192306.0\n15: O 192496.8\n16: P 192339.5\n17: Q 192384.7\n18: R 192320.4\n19: S 192282.8\n20: T 192228.1\n21: U 192101.9\n22: V 192604.1\n23: W 192469.1\n24: X 192297.1\n25: Y 192265.2\n26: Z 192244.2\n    x       V1")

    x       V1
 1: A 192270.8
 2: B 192261.3
 3: C 192292.7
 4: D 191923.5
 5: E 192457.8
 6: F 192240.5
 7: G 192407.9
 8: H 191992.1
 9: I 192348.0
10: J 192436.4
11: K 192248.5
12: L 192270.7
13: M 192280.0
14: N 192306.0
15: O 192496.8
16: P 192339.5
17: Q 192384.7
18: R 192320.4
19: S 192282.8
20: T 192228.1
21: U 192101.9
22: V 192604.1
23: W 192469.1
24: X 192297.1
25: Y 192265.2
26: Z 192244.2
    x       V1


ttt <- system.time(tt <- tapply(DT$v, DT$x, sum));ttt

In [46]:
# 출력
print(" 사용자  시스템 elapsed\n    0.81    0.06    0.90 ")

 사용자  시스템 elapsed
    0.81    0.06    0.90 


 sss <- system.time(ss <- DT[,sum(v),by=x]);sss # 이게 방금 것보다는 더 빠를걸?

In [47]:
# ㅊㄹ
print(" 사용자  시스템 elapsed\n    0.33    0.03    0.21") 

 사용자  시스템 elapsed
    0.33    0.03    0.21


데이터 테이블에서의 by는 tapply에 비해 매우 빠르다.

head(tt)

In [49]:
# 출력
print("       A        B        C        D        E        F\n 192270.8 192261.3 192292.7 191923.5 192457.8 192240.5 ")

       A        B        C        D        E        F
 192270.8 192261.3 192292.7 191923.5 192457.8 192240.5 


head(ss)

In [51]:
# 출력
print("   x       V1\n1: A 192270.8\n2: B 192261.3\n3: C 192292.7\n4: D 191923.5\n5: E 192457.8\n6: F 192240.5")

   x       V1
1: A 192270.8
2: B 192261.3
3: C 192292.7
4: D 191923.5
5: E 192457.8
6: F 192240.5


identical(as.vector(tt),ss$V1)

[1] TRUE

summary / grouping을 x와 y변수로 이용하려는 경우 by = "x,y" 같은 식으로 처리할 수 있다.

sss <- system.time(ss <- DT[,sum(v),by="x,y"]);sss;ss

In [55]:
# 출력
print("사용자  시스템 elapsed\n    0.70    0.02    0.61\n     x y     V1\n  1: A a 7392.969\n  2: A b 7364.329\n  3: A c 7431.784\n  4: A d 7417.803\n  5: A e 7406.803\n ---  \n672: Z v 7350.389\n673: Z w 7437.793\n674: Z x 7376.503\n675: Z y 7398.270\n676: Z z 7423.865")

사용자  시스템 elapsed
    0.70    0.02    0.61
     x y     V1
  1: A a 7392.969
  2: A b 7364.329
  3: A c 7431.784
  4: A d 7417.803
  5: A e 7406.803
 ---  
672: Z v 7350.389
673: Z w 7437.793
674: Z x 7376.503
675: Z y 7398.270
676: Z z 7423.865
